In [9]:
import requests
from requests import Response
from pydantic import BaseModel,Field,RootModel,field_validator
import pandas as pd

class Site(BaseModel):
    站名:str = Field(alias='sna')
    地區:str = Field(alias = 'sarea')
    時間:str = Field(alias = 'mday')
    地點:str = Field(alias = 'ar')
    總車輛數:int = Field(alias='total')
    經度: float = Field(alias='longitude')
    緯度: float = Field(alias='latitude')    
    可借:int = Field(alias='available_rent_bikes')
    可還:int = Field(alias='available_return_bikes')
    狀態:bool = Field(alias='act')
    
    @field_validator("站名")
    @classmethod
    def removefilter(cls,value:str)->str:
        return value.split("_")[-1]
    
class YouBike(RootModel):
    root: list[Site]
    def __iter__(self):
        return iter(self.root)

    def __getitem__(self, item):
        return self.root[item]

try:

    youbike_url = "https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json"
    res:Response = requests.get(youbike_url)
    if res.status_code == 200:
        print("Download Success")
    else:
        print("Download Failure") 
        res.raise_for_status()
except Exception as e:
    print(e)
else:  
    data = YouBike.model_validate_json(res.text)
    all_sites = data.model_dump()
    
df1 = pd.DataFrame(all_sites)

       

Download Success


In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1429 entries, 0 to 1428
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   站名      1429 non-null   object 
 1   地區      1429 non-null   object 
 2   時間      1429 non-null   object 
 3   地點      1429 non-null   object 
 4   總車輛數    1429 non-null   int64  
 5   經度      1429 non-null   float64
 6   緯度      1429 non-null   float64
 7   可借      1429 non-null   int64  
 8   可還      1429 non-null   int64  
 9   狀態      1429 non-null   bool   
dtypes: bool(1), float64(2), int64(3), object(4)
memory usage: 102.0+ KB


如果一次要多個運算函數，就要用agg函數把他們包起來

In [23]:
#df2 = df1.set_index(keys = ['地區'])
#df2.groupby(level='地區')[['總車輛數','可借','可還']].sum()
#df2 = df1.groupby(by='地區')[['總車輛數','可借','可還']].sum()
df2 = df1.groupby(by='地區')[['總車輛數','可借','可還']].agg([('車輛數','sum'),('站點數','count')])
df2


總車輛數         可借         可還     
         車輛數  站點數   車輛數  站點數   車輛數  站點數
地區                                     
中山區     3819  156  1620  156  2176  156
中正區     3209  121  1487  121  1695  121
信義區     3331  106  1321  106  1915  106
內湖區     3499  168  1410  168  2052  168
北投區     2472   94   992   94  1452   94
南港區     2317   92  1148   92  1149   92
士林區     2937  127  1122  127  1756  127
大同區     1629   62   626   62   994   62
大安區     4987  185  2077  185  2827  185
文山區     2203   97   868   97  1332   97
松山區     2343   88   888   88  1425   88
臺大公館校區  1437   53   487   53   911   53
萬華區     2057   80  1031   80  1005   80